![alt text](https://i.imgur.com/1WaY7aA.png)



---



---



# Lab 2.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [29]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [30]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [31]:
print(response.status_code)

200


We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [32]:
parameters = {"lat": -33.87, "lon": 151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [34]:
response = requests.get("http://api.open-notify.org/iss-pass.json", params = parameters)

Print the response header:

In [35]:
print(response.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Mon, 21 Oct 2019 08:25:52 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'X-Cache': 'MISS from SYDULT-PX01', 'X-Cache-Lookup': 'MISS from SYDULT-PX01:3128', 'Via': '1.1 vegur, 1.1 SYDULT-PX01 (squid)', 'Connection': 'keep-alive'}


Print the content of the response (the data that the server returned):

In [36]:
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1571646352, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 607, \n      "risetime": 1571650116\n    }, \n    {\n      "duration": 414, \n      "risetime": 1571656084\n    }, \n    {\n      "duration": 363, \n      "risetime": 1571662020\n    }, \n    {\n      "duration": 560, \n      "risetime": 1571667810\n    }, \n    {\n      "duration": 661, \n      "risetime": 1571673588\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [37]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [48]:
print(response.headers['content-type'])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [49]:
request_sydney = response.json()
print(overheads)

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1571646352, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 607, 'risetime': 1571650116}, {'duration': 414, 'risetime': 1571656084}, {'duration': 363, 'risetime': 1571662020}, {'duration': 560, 'risetime': 1571667810}, {'duration': 661, 'risetime': 1571673588}]}


What kind of object did this give us?

In [53]:
print(type(request_sydney))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [109]:
risetime = request_sydney["response"]
print(risetime)

[{'duration': 607, 'risetime': 1571650116}, {'duration': 414, 'risetime': 1571656084}, {'duration': 363, 'risetime': 1571662020}, {'duration': 560, 'risetime': 1571667810}, {'duration': 661, 'risetime': 1571673588}]


Now extract the `risetime` strings into an array called `srisetimes`

In [110]:
srisetimes = [xrisetime["risetime"] for xrisetime in risetime ]
print(srisetimes)

[1571650116, 1571656084, 1571662020, 1571667810, 1571673588]


```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [111]:
from datetime import datetime

risetimes = []

for xrisetime in srisetimes:
    df_risetime = datetime.fromtimestamp(xrisetime)
    risetimes.append(df_risetime)
    print(df_risetime)

2019-10-21 20:28:36
2019-10-21 22:08:04
2019-10-21 23:47:00
2019-10-22 01:23:30
2019-10-22 02:59:48


Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [112]:
for df_risetime in risetimes:
    print(df_risetime.strftime('%d/%m/%y %I:%M'))

21/10/19 08:28
21/10/19 10:08
21/10/19 11:47
22/10/19 01:23
22/10/19 02:59


Here is an endpoint that tells us who is onboard:

In [113]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [114]:
astronauts = response.json()

In [115]:
print(astronauts)

{'people': [{'name': 'Christina Koch', 'craft': 'ISS'}, {'name': 'Alexander Skvortsov', 'craft': 'ISS'}, {'name': 'Luca Parmitano', 'craft': 'ISS'}, {'name': 'Andrew Morgan', 'craft': 'ISS'}, {'name': 'Oleg Skripochka', 'craft': 'ISS'}, {'name': 'Jessica Meir', 'craft': 'ISS'}], 'number': 6, 'message': 'success'}


In [119]:
print(astronauts['number'])

6


In [120]:
[print(astronaut['name']) for astronaut in astronauts['people']]

Christina Koch
Alexander Skvortsov
Luca Parmitano
Andrew Morgan
Oleg Skripochka
Jessica Meir


[None, None, None, None, None, None]

## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [ ]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [ ]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

>

>

>



---



---


> > > > > > > > >© 2019 Data Science Institute of Australia


---



---

